In [ ]:
# load the dataset into an RDD to get started
input_rdd = sc.textFile("/FileStore/tables/movielens/movies.csv")

### Transformations
Transformations are operations that will not be completed at the time you write and execute the code in a cell - they will only get executed once you have called an action. An example of a transformation might be to convert an integer into a float or to filter a set of values. In this section we will discuss the basic transformations that can be applied on top of RDD.
##### map
The map method is a higher-order method that takes a function as input and applies it to each element in
the source RDD to create a new RDD. The input function to map must take a single input parameter and
return a value.

In [ ]:
# just to show you the first line of the RDD
input_rdd.first()

In [ ]:
# notice how the whole line is one single string
# using the map function, you can transfer the previous RDD to have a list of values instead
input_list = input_rdd.map(lambda line: line.split(','))

In [ ]:
# They are a list object now, instead of pure string
# First is an action. ONLY AT THIS POINT SPARK WILL START PROCESSING
# map is a transformation, which are lazily evaluated. When we executed the map function spark didn't do anything until an action was called upon.
input_list.first()

##### filter
The filter method is a higher-order method that takes a Boolean function as input and applies it to each
element in the source RDD to create a new RDD. A Boolean function takes an input and returns true or
false. The filter method returns a new RDD formed by selecting only those elements for which the input
Boolean function returned true. Thus, the new RDD contains a subset of the elements in the original RDD.

In [ ]:
# the original input RDD has header
input_rdd.take(4)

In [ ]:
# for processing the data I can get rid of the header with a filter operation
movie_info_rdd = input_rdd.filter(lambda line: 'movieId' not in line)

In [ ]:
# we got rid of the header
# again, only at this point spark processes the data
movie_info_rdd.first()

In [ ]:
# lets keep a list rdd for further examples
movie_info_list_rdd = movie_info_rdd.map(lambda x: x.split(','))

##### flatMap
The flatMap method is a higher-order method that takes an input function, which returns a sequence for
each input element passed to it. The flatMap method returns a new RDD formed by flattening this collection
of sequence. The concept and usefulness of a flatMap can be easily explained with the following example:

In [ ]:
# notice the last field of the movie info, you have multiple categories associated with a single movie
movie_info_list_rdd.first()[-1]

In [ ]:
# if we want to do a count each categories appear, we can use flatmap to easily get our answer
# using a flatmap on top of the movie category element causes each entry within the list (categories) to be a single entry/line/tuple in our RDD
movie_cat_rdd = movie_info_list_rdd.flatMap(lambda x: x[-1].split('|'))
movie_cat_rdd.take(10)

In [ ]:
# now we can easily get our category count by using the count by value action (which does exactly what the name suggests).
cat_count = movie_cat_rdd.countByValue()
# we can print out the result in a sorted way using python
sorted(cat_count.items(), key=lambda k_v: k_v[1], reverse=True)

##### Writing Custom Functions
You can write custom functions to process each line within RDD, as illustrated below.

Take the following problem: We want to know what is the oldest movie we have in our dataset. In the movie name the year is given in paranthesis, i.e. Toy Story (1995). We need to extract this year from every name and then convert it to integer. Then we can use the action 'min' to find the minimum value in our data. However, to extract this year, we need to do some special processing. This is where our custom function will come in handy.

In [ ]:
# we will extract the year with this function. if there is problem with our data we will just return a None value
import re
def get_year(name):
    year = None
    try:
      pattern = re.compile(r"\((\d{4})\)")
      year = int(pattern.findall(name)[0])
    except ValueError:
      pass
    except IndexError:
      pass
    
    return year

In [ ]:
# we can use the map operation to apply our custom function to every name in our rdd
movie_year_rdd = movie_info_list_rdd.map(lambda x: get_year(x[1]))

In [ ]:
# we can use the min action to get the oldest movie year. however as you see there was some issue with the data or parsing and we are getting 6 back as our result
movie_year_rdd.filter(lambda x: x is not None).min()

In [ ]:
# so instead of trying to investigate what happened we will simply apply a filter and only consider value above 1000 to get our oldest movie year
movie_year_rdd.filter(lambda x: x is not None).filter(lambda x: x  > 1000).min()

In [ ]:
movie_year_rdd.filter(lambda x: x is not None).max()

#### Read The Docs
Of course these are just the basic, there are many more transformation operations you can do with your RDD. All of them can be found in the official documentaiton: https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD
Here are a few more just to give you some idea:

##### union
The union method takes an RDD as input and returns a new RDD that contains the union of the elements in the source RDD and the RDD passed to it as an input.

```linesFile1 = sc.textFile("...")
linesFile2 = sc.textFile("...")
linesFromBothFiles = linesFile1.union(linesFile2)```

In [ ]:
mammals = sc.parallelize(["Lion", "Dolphin", "Whale"])
aquatics = sc.parallelize(["Shark", "Dolphin", "Whale"])
zoo = mammals.union(aquatics)
zoo.collect()

##### intersection
The intersection method takes an RDD as input and returns a new RDD that contains the intersection of
the elements in the source RDD and the RDD passed to it as an input.

```val linesFile1 = sc.textFile("...")
val linesFile2 = sc.textFile("...")
val linesPresentInBothFiles = linesFile1.intersection(linesFile2)```

In [ ]:
mammals = sc.parallelize(["Lion", "Dolphin", "Whale"])
aquatics = sc.parallelize(["Shark", "Dolphin", "Whale"])
aquaticMammals = mammals.intersection(aquatics)
aquaticMammals.collect()

##### subtract
The subtract method takes an RDD as input and returns a new RDD that contains elements in the source
RDD but not in the input RDD.
```linesFile1 = sc.textFile("...")
linesFile2 = sc.textFile("...")
linesInFile1Only = linesFile1.subtract(linesFile2)```

In [ ]:
mammals = sc.parallelize(["Lion", "Dolphin", "Whale"])
aquatics =sc.parallelize([])
fishes = aquatics.subtract(mammals)
fishes.collect()

##### distinct
The distinct method of an RDD returns a new RDD containing the distinct elements in the source RDD

In [ ]:
sc.parallelize(["Lion", "Dolphin", "Whale","Shark", "Dolphin", "Whale"]).distinct().collect()